In [41]:
pip install import-ipynb

Note: you may need to restart the kernel to use updated packages.


In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.spatial.distance import cdist
import warnings
import import_ipynb
from Utility import PCA_algorithm

warnings.simplefilter(action='ignore', category=FutureWarning)
import random

warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans

In [4]:
data = pd.read_csv(r"datasets/data_by_artist.csv")
data.head()

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,9,0.590111,"""Cats"" 1981 Original London Cast",0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5
1,1,26,0.862538,"""Cats"" 1983 Broadway Cast",0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5
2,1,7,0.856571,"""Fiddler On The Roof” Motion Picture Chorus",0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0
3,1,27,0.884926,"""Fiddler On The Roof” Motion Picture Orchestra",0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0
4,1,7,0.510714,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5


In [5]:
data_for_songs = pd.read_csv(r"datasets/data.csv")
data_for_songs.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [6]:
artist_data = data[['valence', 'artists', 'count', 'popularity']]
artist_data.to_csv('artist_data.csv')

In [7]:
def KMeans_with_PCA_algorithm(song_embedding,n_clusters):
    kmeans_pca = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
    label = kmeans_pca.fit(song_embedding)
    centroids = kmeans_pca.cluster_centers_
    return label, centroids

In [8]:
def filter_based_on_segment(df_segm_pca_kmeans,segment_val,field1,field2):    
    #get list of songs with above song id
    #print(df_segm_pca_kmeans[df_segm_pca_kmeans['Segment K-means PCA']==segment_val].values)
    
    filtered_data_per_segment = df_segm_pca_kmeans[df_segm_pca_kmeans['Segment K-means PCA']==segment_val][[
        field1, field2 ,'com1','com2','Segment K-means PCA']]

    #save to csv file
    filtered_data_per_segment.to_csv('filtered_data_per_segment_for_artists.csv')
    return filtered_data_per_segment

In [9]:
def filter_based_on_cluster_centroid(kmeans_pca, filtered_data_per_segment, segment_val, tracks):
    #calculate distance based on cluster centroid
    scaled_data = kmeans_pca.transform(filtered_data_per_segment[['com1', 'com2']])
    scaled_song_center = kmeans_pca.transform(kmeans_pca.cluster_centers_[segment_val].reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')

    #sort based on distance
    index = list(np.argsort(distances)[0])
    rec_songs = filtered_data_per_segment.iloc[index]

    #recommend based on segment data
    # rec_songs = rec_songs[~rec_songs['name'].isin(filtered_data_per_segment['name'])]
    return rec_songs

In [10]:
p = 0.02  # to randomly select 1% of the rows
df_playlist_spotify = pd.read_csv(r"datasets/spotify_dataset.csv", error_bad_lines=False, warn_bad_lines=False,
                                  skiprows=lambda i: i > 0 and random.random() > p)
df_playlist_spotify.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Miles Kane,Come Closer,IOW 2012
2,9cc0cfd4d7d7885102480dd99e7a90d6,Bruce Springsteen,Death To My Hometown,IOW 2012
3,07f0fc3be95dcd878966b1f9572ff670,Emancipator,Vines,Chill out
4,07f0fc3be95dcd878966b1f9572ff670,Daft Punk,High Fidelity,Daft Punk


In [11]:
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace('"', '')
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace('name', '')
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace(' ', '')
df_playlist_spotify.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

In [12]:
df_playlist = df_playlist_spotify.groupby('artist').filter(lambda x: len(x) >= 50)
print(df_playlist)

                                 user_id             artist  \
2       9cc0cfd4d7d7885102480dd99e7a90d6  Bruce Springsteen   
4       07f0fc3be95dcd878966b1f9572ff670          Daft Punk   
5       07f0fc3be95dcd878966b1f9572ff670           Pendulum   
6       07f0fc3be95dcd878966b1f9572ff670            Ratatat   
9       07f0fc3be95dcd878966b1f9572ff670           Pendulum   
...                                  ...                ...   
257642  2302bf9c64dc63d88a750215ed187f2c         Neil Young   
257643  2302bf9c64dc63d88a750215ed187f2c               Beck   
257645  2302bf9c64dc63d88a750215ed187f2c    Alice In Chains   
257646  2302bf9c64dc63d88a750215ed187f2c       Led Zeppelin   
257647  2302bf9c64dc63d88a750215ed187f2c         Neil Young   

                                 track          playlist  
2                 Death To My Hometown          IOW 2012  
4                        High Fidelity        Daft Punk   
5                          9,000 Miles           Electro  
6      

In [13]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]
print(df_playlist)

                                 user_id                  artist  \
4       07f0fc3be95dcd878966b1f9572ff670               Daft Punk   
5       07f0fc3be95dcd878966b1f9572ff670                Pendulum   
6       07f0fc3be95dcd878966b1f9572ff670                 Ratatat   
9       07f0fc3be95dcd878966b1f9572ff670                Pendulum   
10      07f0fc3be95dcd878966b1f9572ff670                Pendulum   
...                                  ...                     ...   
257502  aedf684d88f71c448add86ee4873278e           NEEDTOBREATHE   
257503  aedf684d88f71c448add86ee4873278e           NEEDTOBREATHE   
257516  aedf684d88f71c448add86ee4873278e                  Eminem   
257517  aedf684d88f71c448add86ee4873278e           NEEDTOBREATHE   
257519  aedf684d88f71c448add86ee4873278e  The Head And The Heart   

                                 track               playlist  
4                        High Fidelity             Daft Punk   
5                          9,000 Miles                E

In [14]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0: 'freq'})[
    ['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
6713,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,61
11334,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,60
43119,fa849dabeb14a2800ad5130907fc5018,Frank Sinatra,40
30613,b1d4116e7cf150ae7d77413620f5f571,Frank Sinatra,39
8239,2e221a9fa50e6487e8a3b7c7ac8c806a,Ennio Morricone,38


In [15]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index': 'artist_id', 0: 'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Frank Sinatra
2,2,Ennio Morricone
3,3,Ella Fitzgerald
4,4,Wolfgang Amadeus Mozart


In [16]:
def GetInPut(user,df_artist):
    inputArtist = pd.DataFrame(user)
    #Filtering out the movies by title
    Id = df_artist[df_artist['artist'].isin(inputArtist['artist'].tolist())]
    #Then merging it so we can get the movieId. It's implicitly merging it by title.
    inputArtist = pd.merge(Id, inputArtist)
    #Dropping information we won't use from the input dataframe
    #inputArtist = inputArtist.drop('year', 1)
    return inputArtist

In [17]:
def get_song_from_artist(artist, n_per_artist= 1):
    list_of_artists_songs = []
    for i in range(len(data_for_songs)):
        if artist in data_for_songs.loc[i, "artists"]:
            list_of_artists_songs.append(data_for_songs.loc[i, ["id", "artists", "name", "popularity"]])

    sorted_list_of_artists_songs = sorted(list_of_artists_songs, key=lambda d: d['popularity'], reverse=True)
    sorted_list_of_songs_only = np.array(sorted_list_of_artists_songs)
    unique_songs = np.unique(sorted_list_of_songs_only[:, 2])
    return unique_songs[:n_per_artist]

In [34]:
from sklearn.decomposition import PCA

def recommend_artists(artist, n=10):
    """
    Recommends songs based on a list of previous songs that a user has listened to.
    """
    song_embedding = PCA_algorithm(artist_data)
    kmeans_pca, centroids = KMeans_with_PCA_algorithm(song_embedding,50)

    df_segm_pca_kmeans = pd.concat([artist_data.reset_index(drop=True), pd.DataFrame(song_embedding)], axis=1)
    df_segm_pca_kmeans.columns.values[-2:] = ['com1', 'com2']
    df_segm_pca_kmeans['Segment K-means PCA'] = kmeans_pca.labels_

    #get segment value of artist
    # artist = 'Frank Sinatra'
    segment_val = df_segm_pca_kmeans[df_segm_pca_kmeans['artists'] == artist]['Segment K-means PCA'].values[0]

    filtered_data_per_segment = filter_based_on_segment(df_segm_pca_kmeans, segment_val,'artists', 'valence')

    rec_artists = filter_based_on_cluster_centroid(kmeans_pca, filtered_data_per_segment, segment_val, artist_data)
    # print(rec_artists)

    #recommend top n artists
    return rec_artists.head(n)['artists'].tolist()

In [35]:
def recommend_artist_and_songs(artist, n_songs=10):
    # get recommended artists
    recommended_artists = pd.DataFrame({
            'artist' : recommend_artists(artist, n_songs)
        })
    # print(recommended_artists)
    recommend_songs_list = pd.DataFrame(columns = ['artist', 'songs'])
    # artist_count = len(recommended_artists)
    # song_per_artist = n_songs > artist_count and (n_songs/artist_count) or 1

    # get songs per artist
    for i in range(len(recommended_artists)):
        artist = recommended_artists.loc[i, "artist"]
        recommend_song = pd.DataFrame({
            'artist' : artist,
            'songs' :get_song_from_artist(artist,data_for_songs, 1)
        })
        recommend_songs_list = recommend_songs_list.append(recommend_song, ignore_index = True)
    return  recommend_songs_list

In [36]:
user = [
    {'artist': 'Ella Fitzgerald', 'freq': 40},
    {'artist': 'Frank Sinatra', 'freq': 10},
    {'artist': 'Lil Wayne', 'freq': 3},
    {'artist': "The Rolling Stones", 'freq': 5},
    {'artist': 'Louis Armstrong', 'freq': 5}
]

In [37]:
inputArtist = GetInPut(user,df_artist).sort_values('freq', ascending=False)
# print(inputArtist)

In [38]:
# pass favorite artist name and no of recommended artist/song you want to listen
recommended_artist_and_songs = recommend_artist_and_songs('Ella Fitzgerald',10)
# this list contains n recommended artist with 1 songs per artist
print(recommended_artist_and_songs)

NameError: name 'np' is not defined

In [62]:
#TODO: refactor and cleanup